In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_nodes, export_model_as_feature
from train import run_experiment

2022-01-12 11:56:47.502738: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load audience overlap edges for level 1

In [2]:
level = 1
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2018_audience_overlap_sites_scrapping_result.json'), level=level)
audience_overlap_sites_NODES = create_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-12 11:56:49 notebooks.utils INFO     Loaded 4238 nodes with records level <= 1 and child size:20335


[('villagevoice.com', 'nylon.com'), ('villagevoice.com', 'slantmagazine.com'), ('villagevoice.com', 'gothamist.com'), ('villagevoice.com', 'screendaily.com'), ('villagevoice.com', 'amny.com')]


In [3]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,villagevoice.com,nylon.com
1,villagevoice.com,slantmagazine.com
2,villagevoice.com,gothamist.com
3,villagevoice.com,screendaily.com
4,villagevoice.com,amny.com


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 11865 Sample: ['ecoffeecup.eco', 'investinrochdale.co.uk', 'rasthaa.com', 'whydontyoutrythis.com', 'kayasthapariwar.weebly.com']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, ecoffeecup.eco to libertyhealthshare.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   8657 non-null   float64
 1   daily_pageviews_per_visitor  8658 non-null   float64
 2   daily_time_on_site           6458 non-null   float64
 3   total_sites_linking_in       11469 non-null  float64
 4   bounce_rate                  6022 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, ecoffeecup.eco to libertyhealthshare.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   11865 non-null  float64
 1   daily_pageviews_per_visitor  8658 non-null   float64
 2   daily_time_on_site           6458 non-null   float64
 3   total_sites_linking_in       11865 non-null  float64
 4   bounce_rate                  6022 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 11865, Edges: 20399

 Node types:
  default: [11865]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [20399]
        Weights: all 1 (default)
        Features: none


# Unsupervised GraphSAGE
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

In [11]:
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 128
epochs = 4
num_samples = [10, 5]

graphsage_link_generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_graphsage_link_gen = graphsage_link_generator.flow(unsupervised_samples)

layer_sizes = [128, 512]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=graphsage_link_generator, bias=True, dropout=0.0, normalize="l2"
)

# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

graphsage_model = keras.Model(inputs=x_inp, outputs=prediction)

graphsage_model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=['acc'],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [12]:
history = graphsage_model.fit(
    train_graphsage_link_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

Epoch 1/4
742/742 [==============================] - 92s 122ms/step - loss: 5.9605e-08 - acc: 0.49986s - loss: 5.9713e-08 - acc: - ETA: 5s - loss: 5.9694e-08 - - ETA: 3s - loss
Epoch 2/4
742/742 [==============================] - 101s 136ms/step - loss: 5.9605e-08 - acc: 0.4998s - loss: 5.9586e-08 - ac - ETA: 6s - loss: 5.9629e-08 - acc: - ETA: 
Epoch 3/4
742/742 [==============================] - 97s 131ms/step - loss: 5.9605e-08 - acc: 0.5000A: 3s - loss: 5.9622e-08 - acc: 0.50 - ETA: 2s - loss: 5.9625e-08 - acc: 0.500 - ETA: 2s - loss: 5.9634e-08 - acc: 0.5 - ETA: 2s - loss: 5.9650e-
Epoch 4/4
742/742 [==============================] - 95s 127ms/step - loss: 5.9605e-08 - acc: 0.499811s - loss:  - ETA: 10s - loss: 5.96 - ETA: 8s - loss: 5.9592e-08 - acc: 0.499 - ETA: 8s - loss: 5.9607e-08 - acc: 0. - ETA - ETA: 2s - loss: 5.9567e-08  - ETA: 1s - loss: 5.9612e-08 - acc: 


In [13]:
embedding_model = keras.Model(inputs=x_inp[0::2], outputs=x_out[0])

from stellargraph.mapper import GraphSAGENodeGenerator

node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_features_df.index)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

# print('Sample:', embeddings_wv['crooked.com'][:10])

93/93 [==============================] - 6s 64ms/step


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, f'graph_sage_audience_overlap_level_{level}_epochs_{epochs}', data_year='2018')

'/Users/panayot/Documents/News-Media-Peers/data/emnlp2018/features/graph_sage_audience_overlap_level_1_epochs_4.json'

In [15]:
run_experiment(features=f'graph_sage_audience_overlap_level_{level}_epochs_{epochs}', dataset='emnlp2018')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| fact |  single classifier  |    combine    |        True        | graph_sage_audience_overlap_level_1_epochs_4 |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 12:03:40 train        INFO     Start training...
01-12 12:03:40 train        INFO     Fold: 0
01-12 12:03:57 train        INFO     Fold: 1
01-12 12:04:09 train        INFO     Fold: 2
01-12 12:04:19 train        INFO     Fold: 3
01-12 12:04:29 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1     |      Accuracy     |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | graph_sage_audience_overlap_level_1_epochs_4 | 40.61284380518628 | 50.51837888784166 | 18.473138548539115 | 0.6795475966069745 |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+


In [16]:
run_experiment(features=f'graph_sage_audience_overlap_level_{level}_epochs_{epochs}', dataset='emnlp2018', task='bias')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| bias |  single classifier  |    combine    |        True        | graph_sage_audience_overlap_level_1_epochs_4 |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 12:04:42 train        INFO     Start training...
01-12 12:04:42 train        INFO     Fold: 0
01-12 12:04:57 train        INFO     Fold: 1
01-12 12:05:07 train        INFO     Fold: 2
01-12 12:05:18 train        INFO     Fold: 3
01-12 12:05:28 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+-------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1      |      Accuracy      |  Flip error-rate  |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+-------------------+--------------------+
| bias |  single classifier  |    combine    |        True        | graph_sage_audience_overlap_level_1_epochs_4 | 36.139472580223035 | 46.842601319509896 | 7.917059377945335 | 0.6107445805843544 |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+-------------------+--------------------+
